In [11]:
import numpy as np
import pandas as pd
import torch_geometric
import networkx as nx
from torch_geometric.data import Data, Dataset
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.to_undirected import ToUndirected
import torch

In [12]:
G = nx.read_edgelist('/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-facebook/raw/facebook/1684.edges', nodetype=int)

In [13]:
edge_array = np.array([(u, v) for u, v in G.edges])
edge_array.sort()

In [14]:
edge_array


array([[2849, 3021],
       [2849, 3335],
       [2849, 3299],
       ...,
       [2691, 2792],
       [3268, 3407],
       [2788, 3271]])

In [15]:
features = np.loadtxt('/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-facebook/raw/facebook/1684.feat')

# sort features by node id
sorted_idx = features[:, 0].argsort()
features = features[sorted_idx, :]


# Map node to index
node_mapper = np.arange(features.shape[0])
node_mapper = dict(zip(features[:, 0].astype(int), node_mapper))

for edge in edge_array:
    edge[0] = node_mapper[edge[0]]
    edge[1] = node_mapper[edge[1]]
    
edge_array

array([[204, 376],
       [204, 690],
       [204, 654],
       ...,
       [ 46, 147],
       [623, 762],
       [143, 626]])

In [16]:
transform = ToUndirected()

# feature 147 will be the sensitive attribute
x = torch.tensor(features[:, 1:], dtype=torch.float32)
y = x[:, 147]

facebook = Data(x=x, y=y, edge_index=torch.tensor(edge_array.T))
facebook = transform(facebook)

In [17]:
x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [18]:
torch.save(facebook, '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-facebook/processed/facebook_1684.pt')